In [4]:
from scoring_code import incr_act_top10
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os, sys

In [7]:
load_dotenv()

SAMPLE_SUBMISSION_ROWS = os.environ.get("SAMPLE_SUBMISSION_ROWS")
TRAINING_DATA = os.environ.get("TRAINING_DATA")

In [6]:
sample_submission_data = pd.read_csv(SAMPLE_SUBMISSION_ROWS)

sample_submission_data.head()

,customer,merchant,predicted_score
0,140,8,0.518033
1,140,120,0.038266
2,140,301,0.308177
3,140,352,0.194978
4,140,692,0.155000


In [18]:
sample_submission_data.shape[0]

1000

In [22]:
import random
import csv


def reservoir_sampling(file_path: str, k: int):
    reservoir = []

    with open(file_path, "r", newline="") as file:
        reader = csv.reader(file)

        # Fill the reservoir with the first k elements
        for row_count, row in enumerate(reader):
            if row_count == 0:
                headers = row
                continue

            if row_count <= k:
                reservoir.append(row)
            else:
                # For subsequent elements, replace a random element with probability k / row_count
                probability = k / row_count
                if random.random() < probability:
                    replace_index = random.randint(0, k - 1)
                    reservoir[replace_index] = row

    return reservoir, headers


SIZE = 1000

rows, headers = reservoir_sampling(file_path=TRAINING_DATA, k=SIZE)

training_df = pd.DataFrame(rows, columns=headers)

training_df.head()

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
0,0,0,,85.06795454545453,27.145000000000003,9.0,271.45000000000005,10.0,9.0,10.0,...,,,0.8461538461538461,167.0,52.56864359915679,0.5,75.0,1.38471397524816,167250,490938
1,0,0,,49.86344827586206,,,,,,8.0,...,4077.0,10781.0,1.0,151.595,74.38684249537502,12.333333333333334,165.0,2.522088684126093,231078,129407
2,0,0,,46.38625,21.9,1.0,21.9,1.0,1.0,1.0,...,5496.0,10611.0,0.675,35.38,94.472931221417,1.8333333333333333,236.0,2.331255218569399,130007,206750
3,0,0,,50.5432,38.41333333333333,5.0,230.48,6.0,6.0,6.0,...,,,0.7777777777777778,1043.5,14.080650815038076,0.6666666666666666,101.0,1.769063618679541,223288,164881
4,0,0,,422.4774285714286,,,,,,16.0,...,,,0.9545454545454546,32.54,79.16119107406472,9.833333333333334,43.0,25.80276339375133,314567,330922


In [23]:
training_df.shape[0]

1000

In [24]:
training_df["predicted_score"] = sample_submission_data["predicted_score"].values

training_df.head()

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,predicted_score
0,0,0,,85.06795454545453,27.145000000000003,9.0,271.45000000000005,10.0,9.0,10.0,...,,0.8461538461538461,167.0,52.56864359915679,0.5,75.0,1.38471397524816,167250,490938,0.518033
1,0,0,,49.86344827586206,,,,,,8.0,...,10781.0,1.0,151.595,74.38684249537502,12.333333333333334,165.0,2.522088684126093,231078,129407,0.038266
2,0,0,,46.38625,21.9,1.0,21.9,1.0,1.0,1.0,...,10611.0,0.675,35.38,94.472931221417,1.8333333333333333,236.0,2.331255218569399,130007,206750,0.308177
3,0,0,,50.5432,38.41333333333333,5.0,230.48,6.0,6.0,6.0,...,,0.7777777777777778,1043.5,14.080650815038076,0.6666666666666666,101.0,1.769063618679541,223288,164881,0.194978
4,0,0,,422.4774285714286,,,,,,16.0,...,,0.9545454545454546,32.54,79.16119107406472,9.833333333333334,43.0,25.80276339375133,314567,330922,0.155000


In [25]:
result = incr_act_top10(input_df=training_df, pred_col="predicted_score")

result

   ind_recommended  avg_30d_act
0                0     0.005682
1                1     0.000000


/Users/wayne/src/AMEX AI Hackathon/scoring_code.py:39: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col] == 1, "avg_30d_act"])
/Users/wayne/src/AMEX AI Hackathon/scoring_code.py:40: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  not_recommended_avg_30d_act = float(


-0.005681818181818182